In [ ]:
# install asteroid
!git clone https://github.com/asteroid-team/asteroid
%cd asteroid
!pip install -e .
!pip install asteroid

!pip install pytorch-lightning

In [ ]:
import torch
from torch import optim
from pytorch_lightning import Trainer
from asteroid.models import ConvTasNet
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper
from asteroid.data import LibriMix
from asteroid.engine import System
from torch.utils.data import DataLoader

# to enable cuda, you have to do a manual toggle in the settings
# menu 'runtime->change runtime->hardware accelerator->gpu'
torch.cuda.is_available()

In [ ]:
# import custom data set from py in google drive
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive')
from custom_dataset_sandbox import CustomMixture 

In [ ]:
csv_path = '/content/drive/MyDrive/output/'
%cd /content/drive/MyDrive/output/
train_loader = DataLoader(CustomMixture(csv_path, "truetrain_metadata.csv"), num_workers=2, batch_size=1)
val_loader = DataLoader(CustomMixture(csv_path, "trueval_metadata.csv"), num_workers=2, batch_size=1)

In [ ]:
model = ConvTasNet(n_src=2)

# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [ ]:
trainer = Trainer(max_epochs=10, gpus=1, auto_select_gpus=True, log_every_n_steps=1)
trainer.fit(system)

In [ ]:
# test_loader = DataLoader(CustomMixture(csv_path, "truetest_metadata.csv"), num_workers=2)
# model.eval()

In [ ]:
# import librosa
# audiofile, _ = librosa.load('/content/drive/MyDrive/output/mixture/mixtureclip0.wav')
# prediction = model(torch.from_numpy(audiofile))
#trainer.test(model, dataloaders=test_loader)
#print(prediction)

In [ ]:
# import torch
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.insert(0,'/content/drive/My Drive')
# torch.cuda.is_available()
# checkpoint = torch.load('/content/drive/MyDrive/output/lightning_logs/version_2/checkpoints/epoch=0-step=11467.ckpt')
# torch.cuda.is_available()

# #print(checkpoint.keys())
# model = ConvTasNet(n_src=2)
# # model.from_pretrained(ConvTasNet, checkpoint['state_dict'])
# model.load_state_dict(checkpoint['state_dict'])